## Agent执行过程

In [1]:
# 导入API
import os
import api_key
os.environ['TAVILY_API_KEY']=api_key.TAVILY_API_KEY

In [2]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain_community.tools.tavily_search import TavilySearchResults

from langchain_ollama import ChatOllama
# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:latest"
model  = ChatOllama(model=model_name,base_url='http://192.168.3.155:11434')

# 定义工具
search = TavilySearchResults(max_results=2)
tools = load_tools(
    ["llm-math"], 
    llm=model #第一步初始化的模型
)
tools.append(search)

# 初始化代理
agent= initialize_agent(
    tools, #第二步加载的工具
    model, #第一步初始化的模型
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,  #代理类型
    handle_parsing_errors=True, #处理解析错误
    verbose = True #输出中间步骤
)

C:\Users\wushaogui\AppData\Local\Temp\ipykernel_34908\3218198488.py:20: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent= initialize_agent(


In [3]:
agent("计算300的25%") 

C:\Users\wushaogui\AppData\Local\Temp\ipykernel_34908\1810155123.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent("计算300的25%")




> Entering new AgentExecutor chain...
Question: 计算300的25%
Thought: 这个问题需要进行数学运算来得出答案。25%可以转换为小数形式，即0.25。因此，我们需要将300乘以0.25。
Action:
```
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```

Observation: Answer: 75.0
Thought:我得出了300的25%是75。

Final Answer: 75

> Finished chain.


{'input': '计算300的25%', 'output': '75'}

In [4]:
agent("什么是智能体agent") 



> Entering new AgentExecutor chain...
Question: 什么是智能体agent

Thought: 这是一个关于智能体（agent）概念的问题。通常在计算机科学、人工智能领域，智能体指的是具有感知并执行任务能力的实体。

Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "什么是智能体agent"
}
```
Observation: [{'url': 'https://baike.baidu.com/item/智能体/9446647', 'content': '智能体（Agent）是指能够感知环境并采取行动以实现特定目标的代理体。 它可以是 软件 、 硬件 或一个 系统，具备自主性、适应性和交互能力。 智能体通过感知环境中的变化（如通过传感器或数据输入），根据自身学习到的知识和算法进行判断和决策，进而执行动作以影响环境或达到预定的目标。 智能体在 人工智能 领域广泛应用，常见于 自动化系统 、 机器人 、虚拟助手和游戏角色等，其核心在于能够自主学习和持续进化，以更好地完成任务和适应复杂环境。 智能体是指能够感知环境并自主采取行动以实现特定目标的实体。'}, {'url': 'https://cloud.tencent.com/developer/article/2422923', 'content': '智能体是什么. 智能体的英文是 Agent，AI 业界对智能体提出了各种定义。个人理解，智能体是一种通用问题解决器。从软件工程的角度看来，智能体是一种基于大语言模型的，具备规划思考能力、记忆能力、使用工具函数的能力，能自主完成给定任务的计算机程序。'}]
Thought:Action:
```
{
  "action": "tavily_search_results_json",
  "action_input": "什么是智能体agent"
}
```


Observation: [{'url': 'https://baike.baidu.com/item/智能体/9446647', 'content': '智能体（Agent）是指能够感知环境并采取行动以实现特定目标的代理体。它可以是软件、硬件或一个系统，具备自主性、适

{'input': '什么是智能体agent',
 'output': '在人工智能领域中，智能体（Agent）是指能够感知环境并采取行动以实现特定目标的代理体。它可以是软件、硬件或一个系统，具备自主性、适应性和交互能力。智能体通过感知环境中的变化（如通过传感器或数据输入），根据自身学习到的知识和算法进行判断和决策，进而执行动作以影响环境或达到预定的目标。\n\n从软件工程的角度来看，智能体是一种基于大语言模型的，具备规划思考能力、记忆能力、使用工具函数的能力，并能自主完成给定任务的计算机程序。'}

## agent高级使用

In [5]:
# Import relevant functionality
from langchain.utilities import OpenWeatherMapAPIWrapper
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults

import requests
from api_key import GAODE_WEATHER_API_KEY
from langchain.agents import tool

@tool
def get_weather(city_name:str)->dict:
    '''根据输入的城市名，使用高德接口查询当地天气'''
    # 构建请求URL
    url = f'https://restapi.amap.com/v3/weather/weatherInfo?key={GAODE_WEATHER_API_KEY}&city={city_name}&extensions=all'
    
    try:
        # 发送GET请求
        response = requests.get(url)
        # 检查请求是否成功
        if response.status_code == 200:
            # 解析响应内容
            data = response.json()
            # 检查API响应状态
            if data.get('status') == '1' and data.get('infocode') == '10000':
                return data  # 返回天气数据
            else:
                return {}  # 返回空结果，因为API返回了错误信息
        else:
            return {}  # 返回空结果，因为HTTP状态码不是200
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}  # 返回空结果，因为请求过程中发生了异常
search = TavilySearchResults(max_results=2)

from langchain_ollama import ChatOllama
# 初始化Ollama LLM，注意需要后台开启ollama服务
model_name = "qwen2.5:latest"
model  = ChatOllama(model=model_name,base_url='http://192.168.3.155:11434')

# Create the agent
memory = MemorySaver()
tools2 = [search,get_weather]
agent_executor = create_react_agent(model, tools2, checkpointer=memory)

In [6]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="你好，我是王五，我住在广州")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='你好，王五！你在广州生活一定很愉快。想查询一下广州当前的天气情况吗？如果不想查天气的话，我们也可以聊其他话题。', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2024-12-02T00:52:57.316147059Z', 'done': True, 'done_reason': 'stop', 'total_duration': 594793341, 'load_duration': 30257995, 'prompt_eval_count': 251, 'prompt_eval_duration': 77092000, 'eval_count': 35, 'eval_duration': 337965000, 'message': Message(role='assistant', content='你好，王五！你在广州生活一定很愉快。想查询一下广州当前的天气情况吗？如果不想查天气的话，我们也可以聊其他话题。', images=None, tool_calls=None)}, id='run-a5ba18b5-f766-42d5-8491-2309ce957961-0', usage_metadata={'input_tokens': 251, 'output_tokens': 35, 'total_tokens': 286})]}}
----


In [7]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="明天我准备去北京旅游，适合去哪里游玩")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2024-12-02T00:53:03.732086229Z', 'done': True, 'done_reason': 'stop', 'total_duration': 664620284, 'load_duration': 31383244, 'prompt_eval_count': 305, 'prompt_eval_duration': 45418000, 'eval_count': 30, 'eval_duration': 289704000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='tavily_search_results_json', arguments={'query': '北京 明天 旅游景点'}))])}, id='run-51b7c922-359e-4b3e-b074-8a0a7ed5e32d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '北京 明天 旅游景点'}, 'id': '1fe8d1de-d78c-471d-ad00-832e7495f50e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 305, 'output_tokens': 30, 'total_tokens': 335})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://you.ctrip.com/sight/beijing1.html", "content": "北京旅游景点大全，携程攻略社区! 告诉您北京有哪些热门旅游景点及旅游必去景点，提供北京旅游景点介绍、图片、门票、点评、景点排

In [8]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="我准备去上海旅游3天，帮我指定一个行程单，首先查询当地的天气，如果下雨，请安排室内项目，如果晴朗，优先安排室外项目")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2024-12-02T00:53:25.615388721Z', 'done': True, 'done_reason': 'stop', 'total_duration': 863429075, 'load_duration': 108520424, 'prompt_eval_count': 539, 'prompt_eval_duration': 152138000, 'eval_count': 41, 'eval_duration': 401491000, 'message': Message(role='assistant', content='', images=None, tool_calls=[ToolCall(function=Function(name='get_weather', arguments={'city_name': '上海'}))])}, id='run-b468291d-c48d-4181-8b68-b710c281ead3-0', tool_calls=[{'name': 'get_weather', 'args': {'city_name': '上海'}, 'id': '1700eab4-b74c-412a-8307-dc95d2fd6807', 'type': 'tool_call'}], usage_metadata={'input_tokens': 539, 'output_tokens': 41, 'total_tokens': 580})]}}
----
{'tools': {'messages': [ToolMessage(content='{"status": "1", "count": "1", "info": "OK", "infocode": "10000", "forecasts": [{"city": "上海市", "adcode": "310000", "province": "上海", "reporttime": "2024-12-02 08:33